In [1]:
import sys
import os
import subprocess
import zipfile
import numpy as np

from oodles import Framework
from oodles import Signal
from oodles import monitor
from oodles import ModelSignal, AnnotationMethod, Anomaly

from dataset import input_to_dataset_transformation, read_json, write_json
from model import run_real_world_inference, get_accuracy
from pushup_signal import pushup_signal
from train import train_model
from contextlib import redirect_stdout

orig_training_file = 'data/training_data.json'

In [2]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
if not os.path.exists(data_dir):
    try:
        file_downloaded_ok = subprocess.check_output("wget " + remote_url, shell=True)
    except:
        print("Could not load training data")
    with zipfile.ZipFile("data.zip", 'r') as zip_ref:
        zip_ref.extractall("./")

    full_training_data = read_json(orig_training_file)
    np.random.seed(1)
    np.random.shuffle(full_training_data)
    reduced_training_data = full_training_data[0:1000]
    write_json(orig_training_file, reduced_training_data)

In [3]:
real_world_test_cases = 'data/real_world_testing_data.json'
data_save_fold_name = 'oodles_smart_data'
my_signal = (Signal("Pushup", pushup_signal) | Signal(ModelSignal.BINARY_ENTROPY_CONFIDENCE, 
                is_model_signal=True, extra_args={'conf_threshold': 0.8}))
golden_testing_file = 'data/golden_testing_data.json'
annotation_args = {'master_file': 'data/master_annotation_data.json'}

In [4]:
cfg = {
    # Define your signal to identify edge cases
    "checks": [{
        'type': Anomaly.EDGE_CASE, 
        "signal_formulae": (Signal("Pushup", pushup_signal) | Signal(ModelSignal.BINARY_ENTROPY_CONFIDENCE, 
                is_model_signal=True, extra_args={'conf_threshold': 0.8}))}],

    # Connect training pipeline to annotate data and retrain the model
    "training_args": {
        "data_transformation_func": input_to_dataset_transformation,  
        "annotation_method": {"method": AnnotationMethod.MASTER_FILE, "args": annotation_args}, 
        "training_func": train_model, 
        "fold_name": data_save_fold_name,  
        "orig_training_file": orig_training_file,  
    },

    # Connect evaluation pipeline to test retrained model against original model
    "evaluation_args": {
        "inference_func": get_accuracy,
        "golden_testing_dataset": golden_testing_file,
        "metrics_to_check": ['accuracy']
    }
}

framework = Framework(cfg)

@monitor(framework)
def model_predict(args):
    with open('evaluation_logs.txt', 'w') as f:
        with redirect_stdout(f):
            return args['model'].predict(args['kps'])

Deleting the folder:  oodles_smart_data


In [5]:
train_model('data/training_data.json', 'initial_model')

Training on:  data/training_data.json  which has  1000  data-points


2022-11-12 00:06:43.875474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
3/3 [==============================] - 1s 160ms/step - loss: 144.1491 - binary_accuracy: 0.2969
Epoch 2/10
3/3 [==============================] - 0s 163ms/step - loss: 132.3618 - binary_accuracy: 0.2734
Epoch 3/10
3/3 [==============================] - 1s 194ms/step - loss: 120.7517 - binary_accuracy: 0.3047
Epoch 4/10
3/3 [==============================] - 0s 161ms/step - loss: 115.5801 - binary_accuracy: 0.2904
Epoch 5/10
3/3 [==============================] - 0s 154ms/step - loss: 101.2244 - binary_accuracy: 0.3177
Epoch 6/10
3/3 [==============================] - 0s 161ms/step - loss: 92.0054 - binary_accuracy: 0.3021
Epoch 7/10
3/3 [==============================] - 0s 160ms/step - loss: 87.5556 - binary_accuracy: 0.3242
Epoch 8/10
3/3 [==============================] - 0s 150ms/step - loss: 80.8414 - binary_accuracy: 0.3320
Epoch 9/10
3/3 [==============================] - 0s 163ms/step - loss: 73.7936 - binary_accuracy: 0.3685
Epoch 10/10
3/3 [========================

In [6]:
run_real_world_inference(real_world_test_cases, 'initial_model', model_predict)

50  edge-cases collected out of  179  inferred samples
100  edge-cases collected out of  392  inferred samples
150  edge-cases collected out of  593  inferred samples
200  edge-cases collected out of  805  inferred samples
250  edge-cases collected out of  958  inferred samples
Kicking off re-training
251 data-points selected out of 960
Training on:  oodles_smart_data/1/training_dataset.json  which has  2255  data-points
Epoch 1/10
8/8 [==============================] - 1s 163ms/step - loss: 27.2824 - binary_accuracy: 0.7134
Epoch 2/10
8/8 [==============================] - 1s 163ms/step - loss: 16.5457 - binary_accuracy: 0.6152
Epoch 3/10
8/8 [==============================] - 1s 160ms/step - loss: 13.0344 - binary_accuracy: 0.5811
Epoch 4/10
8/8 [==============================] - 1s 164ms/step - loss: 10.0725 - binary_accuracy: 0.6934
Epoch 5/10
8/8 [==============================] - 1s 161ms/step - loss: 7.0817 - binary_accuracy: 0.7354
Epoch 6/10
8/8 [==============================